In [ ]:
import requests
from typing import TypedDict, List, Optional, Dict
from langgraph.graph import StateGraph, END

In [ ]:
# The STATE

class DealState(TypedDict):
    search_query: str
    target_price: float
    
    # Internal Data
    game_id: Optional[str] # The API's specific ID for the game
    game_title: Optional[str] # The API's official title for the game
    deals: List[Dict] # Raw list of prices from all stores
    best_deal: Optional[Dict] # The best deal found

    # Output Data
    decision: str # "buy", "wait", "error"
    message: str # Final report

### The NODES - Search game, fetch price, analyzer, notifier

In [ ]:
# Node 1 - Search Game

def search_game_node(state: DealState):
    """
    Step 1: Real API call to resolve the user's query into a Game ID.
    """
    query = state['search_query']
    print(f"--- 1. SEARCHING: Looking for '{query}' in public database... ---")
    
    # REAL DATA SOURCE: CheapShark API
    url = f"https://www.cheapshark.com/api/1.0/games?title={query}&limit=1"
    try:
        response = requests.get(url)
        data = response.json()
        
        if not data:
            return {"decision": "error", "message": f"Game '{query}' not found."}
            
        # Extract the official ID and Title
        game_id = data[0]['gameID']
        official_title = data[0]['external']
        
        return {"game_id": game_id, "game_title": official_title}
        
    except Exception as e:
        return {"decision": "error", "message": f"API Error: {str(e)}"}



In [ ]:
# Node 2 - Fetch Price

def fetch_prices_node(state: DealState):
    """
    Step 2: Uses the ID from Step 1 to fetch live pricing from 25+ stores.
    """
    if state.get("decision") == "error":
        return {} # Skip if previous step failed
        
    game_id = state['game_id']
    print(f"--- 2. FETCHING: Getting live prices for ID {game_id}... ---")
    
    # REAL DATA SOURCE: Fetching specific deal details
    url = f"https://www.cheapshark.com/api/1.0/games?id={game_id}"
    try:
        response = requests.get(url)
        data = response.json()
        
        # 'deals' is a list of prices from Steam, Epic, etc.
        all_deals = data.get('deals', [])
        return {"deals": all_deals}
        
    except Exception as e:
        return {"decision": "error", "message": f"Price Fetch Error: {str(e)}"}
    
    

In [ ]:
# Node 3 - analyzer

def analyzer_node(state: DealState):
    """
    Step 3: Busines Logic. Find the cheapest price and compares to target. 
    """
    if state.get("decision") == "error":
        return {} # Skip if previous step failed
    
    deals = state["deals"]
    target = state["target_price"]

    if not deals:
        return {"decision": "wait", "message": "No deals found"}

    # Logic: Sort by price (ascending) to fid the absolute cheapest price. 
    # Real world data comes as strings, need to cast to float

    sorted_deals = sorted(deals, key=lambda x: float(x['price']))
    cheapest_deal = sorted_deals[0]

    current_price = float(cheapest_deal['price'])
    store_id = cheapest_deal['storeID']

    print(f"--- 3. ANALYZING: Found dest deal at ${current_price} at store {store_id}... ---")
   
    # Make a decision based on the price
    if current_price <= target:
       return {
        "decision": "buy", 
        "best_deal": cheapest_deal,
        "message": f"Found deal at ${current_price} at store {store_id}"
        }
    else:
       return {
        "decision": "wait", 
        "best_deal": cheapest_deal,
        "message": "Wait for a better deal. All current deals are too expensive. Best is ${current_price} at store {store_id}, but you want ${target}"
        }

In [ ]:
# Node 4 - Notifier 

def notifier_node(state: DealState):
    """
    Step 4: Notify the user if the price is below the target price
    """
    color = "\033[92m" if state['decision'] == 'buy' else "\033[93m" # Green or Yellow
    reset = "\033[0m"
    
    print(f"\n{color}>>> NOTIFICATION: {state['message']} <<<{reset}\n")
    return {}

In [ ]:
# Graph Construction

workflow = StateGraph(DealState)

# Add Nodes
workflow.add_node("search", search_game_node)
workflow.add_node("fetch", fetch_prices_node)
workflow.add_node("analyze", analyzer_node)
workflow.add_node("notify", notifier_node)

In [ ]:
# Add edges
workflow.set_entry_point("search")

# Conditional Routing from search
# If search fails, go straight to notify. If valid, go to fetch.
def check_search_success(state):
    if state.get("decision") == "error":
        return "notify"
    return "fetch"

workflow.add_conditional_edges(
    "search",
    check_search_success,
    {
        "notify": "notify",
        "fetch": "fetch"
    }
)

workflow.add_edge("fetch", "analyze")
workflow.add_edge("analyze", "notify")
workflow.add_edge("notify", END)

In [ ]:
# Compile

app = workflow.compile()

In [ ]:
# Real life simulation

# We want "Batman" for under $5.
print("--- RUN 1. Searching for a cheap Batman game...")

app.invoke({"search_query": "Batman Arkham Knight", "target_price": 5.00, "decision": "pending"})